# **<center>EE435 Final Project: 3D human pose estimation in 2D video</center>**

**<center>Group Members: Juntao He, Yujie Jiang, Yuqiu Huang, Zhisheng Lin</center>**
**<center>Video Introduction: https://www.youtube.com/watch?v=3BItkA4xuI4&feature=youtu.be</center>**

---

## **1.Topic**
Human Pose Estimation is about estimating 2D and 3D localization of human joints in images or videos. In our project, we perform 3D human pose estimation from 2D videos. Given a 2D video of a person, the main idea of 3D pose estimation is to detect 3D locations of the person's joints, which form a “skeleton”. It has applications in human action recognition, motion capture, interesting mobile applications, sport, augmented and virtual reality, robotics, etc. It is an important step towards understanding people in images and videos. We realized this process through 2 steps, one is using Detectron2 to detect the 2D joint keypoints from an arbitrary 2D video, the other one is applying [a pre-trained temporal CNN](https://github.com/facebookresearch/VideoPose3D) for predicting 3D joint keypoints from 2D keypoints.

Our model was trained based on the Human3.6M dataset, which is a 3D human pose dataset containing 3.6 million human poses and corresponding images. It is easy to download, extract, and preprocess the images and annotations from Human3.6M. We can get the 3D pose estimation output as the Human3.6M format. The following image shows the 17 keypoints of human joints defined in Human3.6M.

<img src="img/Human3.6M.jpg" width="30%">

## **2. Pipeline**

![TCNN1.png](img/Pipeline.jpg)

The above figure is the pipeline of our project. In the first step, Detectron2 is used to extract the 2D joint keypoints from an arbitrary 2D video. Then we apply the temporal CNN for predicting 3D joint keypoints from 2D keypoints

## **3. Methodology**

### **3.1 Detectron2**
The first step of the pipeline is to generate 2D keypoints from 2D videos using Detectron2. Detectron2 is Facebook AI Research's next generation software system that implements state-of-the-art object detection algorithms. It is a ground-up rewrite of the previous version, Detectron, and it originates from maskrcnn-benchmark.

There are three steps, which are configuration, predicting and dataset creating, in generating 2D keypoints from 2D videos using Detectron2. In configuration step, we just do some preparing job, like downloading the Detectron2 model and setting up some parameters in yaml files, then put the video that we used in a specific folder. In predicting step, we infer the 2D keypoints by running infer.sh file and get the output as custom NumPy archives. In dataset creating step, we finally get an output dataset of 2D keypoints with COCO format as the input of the temporal CNN.

### **3.2 Temporal CNN**
Temporal CNN model is a fully convolutional architecture with residual connections that takes a sequence of 2D poses as input and transforms them through temporal convolutions. (In this part, we download the pretrained model from [VideoPose3D](https://github.com/facebookresearch/VideoPose3D) project as the model is too hard to train.)

<img src="img/TCNN1.jpg" width="70%">

The following image is an instantiation of temporal CNN, which shows how the temporal CNN in our project works. A temporal dilated convolutional model is used. The input of 2D keypoints are shown in parentheses where (243, 34) denotes 243 frames and 34 channels. Convolutional layers are in green where ($2J$, 3d1, 1024) denotes $2J$ ($J=17$, which means the number of joints defined in Human3.6M) input channels, kernels of size 3 with dilation 1, and 1024 output channels.  The aim of slicing the residuals is to match the shape of subsequent tensors. Through nine blocks of neural network, we finally get one frame by a 51-channel output, which represents the 3D coordinates of 17 joints.

<img src="img/model.png" width="80%">

Temporal CNN has some advantages that RNNs do not have. Convolutional models enable parallelization over both the batch and the time dimension while RNNs cannot be parallelized over time. In addition, in CNNs, the length of the gradient path between the output and the input is fixed, regardless of the length of the sequence, thereby mitigating the vanishing and exploding gradients found in RNNs. A convolutional architecture also offers precise control over the temporal receptive field, which is beneficial to model temporal dependencies for the task of 3D pose estimation. Moreover, employing dilated convolutions to model long-term dependencies while at the same time maintaining efficiency.

## **4. Results**

Based on the methodologies above, the pre-trained temporal CNN is able to make good 3D pose estimations from 2D keypoints. Under most situations, the estimations fits human intuition well. For example, the image below shows a sequence of images from a video of a basketball player, the extracted 2D keypoints, and the reconstructed 3D poses given by our model.

<img src="img/result-basketball.png" width="70%">

We also got other interesting results. This image gives the 3D pose estimations for an actor(Bruce Lee) practicing kungfu.

<img src="img/result-kungfu.png" width="70%">

Here is another result of one of our team member playing soccer.

<img src="img/result-soccer.png" width="70%">

## **5. Conclusion**

**Dataset**: Human 3.6M  
**Model for 2D keypoints extraction**: RCNN and FPN (in Detectron2)   
**Model for 3D keypoints prediction**: Temporal CNN  
**Dependencies**:  
- Python 3+ distribution
- Pytorch 1.4
- Nvidia driver 435
- Cuda 10.1 + cuDNN 7.6.5  

**System**: Ubuntu 18.04

### **5.1 Shortcomings**

Though in most cases our model could give relatively reasonable predictions of 3D poses, currently there are still some limitations in certain situations:

- **Inability for multiple persons**.   
The model is developed based on the assumption of a single person. No adaptation has been made for multiple persons. To solve this problem in the future, we could apply some other models to split different persons from the video before running Detectron2.
- **Instable estimations in noisy backgrounds**.   
Given a video with a messy background, it might be difficult to distinguish the person and thus our model would give instable and inaccurate predictions. The following figure shows an example of this problem, where the 2D keypoints are sometimes fixed onto the background audience instead of the basketball player.

<img src="img/instability.png" width="33%">

### **5.2 Future Applications**

As a method for human pose estimation, our project also have multiple future applications in action recognition, motion capture, sports, virtual reality, etc. For example, one possible application is to help with the training of athletes or sports lovers. With accurate 3D pose estimation, they can receive the guidance from professional coaches or well-programmed computers to correct their gestures. Remote training might also be available in the future.